<a href="https://colab.research.google.com/github/Chamanthi000/Chamanthi000.github.io/blob/main/smlass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from __future__ import division
import sys

In [31]:
def convertdattoframe(v):  # a function to convert data for dataframe
  xtrain = [[element  for element in upperElement] for upperElement in v['X_trn']]
  #xtest = [[element  for element in upperElement] for upperElement in v['X_tst']]
  ytrain = [[element  for element in upperElement] for upperElement in v['Y_trn']]
  #ytest = [[element  for element in upperElement] for upperElement in v['Y_tst']]
  return xtrain,ytrain

In [32]:
def createdataframe(x,y): #a function to create dataframes
  df=pd.DataFrame(x,columns=['x1','x2'])
  df['y']=np.array(y).flatten()
  #df['intercept']=np.ones(df['x1'].shape[0])
  return df

In [33]:
def converttonumpy(df_train): #to convert dataframes to numpy form
  x1 = df_train[['x1']].to_numpy()
  x2 = df_train[['x2']].to_numpy()
  y=df_train[['y']].to_numpy()
  
  return x1,x2,y

In [5]:
from google.colab import files #library to import files

In [34]:
upload=files.upload()

Saving data1.mat to data1 (10).mat


In [35]:
#!pip install -U -q scipy
# Load the data
from scipy import io
v = io.loadmat('data1.mat')

In [36]:
# data preprocessing to find weights
xtrain,ytrain=convertdattoframe(v)
df_train=createdataframe(xtrain,ytrain)

#x1,x2,y=converttonumpy(df_train,df_test)

In [37]:
def totentropy(df_train):
  y_0=(df_train["y"] == 0).sum()
  y_1=(df_train["y"] == 1).sum()
  

  k_0=(y_0/(y_0 + y_1))
  k_1=(y_1/(y_0 + y_1))
  if k_0!=0 and k_1!=0:
    entropy=(k_0 * (math.log(1/k_0)))+k_1 * (math.log(1/k_1))
  elif k_0==0:
    entropy=k_1 * (math.log(1/k_1))
  else:
    entropy=k_0 * (math.log(1/k_0))
 
  return entropy

  

In [38]:
actualentropy=totentropy(df_train)

In [11]:
def informationgain(feature_1,feature_1_df):
  feature_1_df['l']=""
  for i in range (0,20):
    key=feature_1_df.x1.iloc[i]
    df_1 = feature_1[feature_1['x1'] <=key]
    df_2 = feature_1[feature_1['x1'] >key]
    
    y_df1=(df_1["y"]).sum()
    y_df2=(df_2["y"]).sum()
    
    k_df1=(y_df1/(y_df1+y_df2))
    k_df2=(y_df2/(y_df1 + y_df2))
    if k_df1!=0 and k_df2!=0:
      ee=(k_df1 * totentropy(df_1) +k_df2 * totentropy(df_2))
    elif k_df1==0:
      ee=k_df2 * totentropy(df_2)
    else:
      ee=k_df1 * totentropy(df_1)

    ifg=actualentropy-ee
    index_f=feature_1_df.index[feature_1_df['x1'] == key].tolist()[0]
    
    feature_1_df.loc[index_f,'l'] =ifg
  return feature_1_df

In [12]:
feature_1=pd.DataFrame(df_train,columns=['x1','y']).sort_values(by = 'x1')
feature_1_df=pd.DataFrame(df_train,columns=['x1','y']).sort_values(by = 'x1').iloc[::7]
feature_1_infogain=informationgain(feature_1,feature_1_df)

In [13]:
feature_2=pd.DataFrame(df_train,columns=['x2','y']).sort_values(by = 'x2')
feature_2_df=pd.DataFrame(df_train,columns=['x2','y']).sort_values(by = 'x2').iloc[::7]
feature_2=feature_2.rename(columns={'x2': 'x1'})  
feature_2_df=feature_2_df.rename(columns={'x2': 'x1'})  
feature_2_infogain=informationgain(feature_2,feature_2_df)


In [14]:
def findbestfeaturevalue(feature_1_infogain):
  bfig1=feature_1_infogain["l"].max()
  bfindex1=feature_1_infogain.index[feature_1_infogain['l'] == bfig1].tolist()[0]
  bestfeature_1=feature_1_infogain.x1[bfindex1]
  return bestfeature_1

In [15]:
def comparebestfeature(x,y):
  if x>y:
    print("feature-1 gives best split with feature value "+str(x)+  "  and with best information gain of " , feature_1_infogain["l"].max())
  else:
    print("feature-2  gives best split with feature value"  + str(y) + "and with best information gain of " , feature_2_infogain["l"].max())

In [16]:
bestfeature_1value=findbestfeaturevalue(feature_1_infogain)

In [17]:
bestfeature_2value=findbestfeaturevalue(feature_2_infogain)

In [18]:
print("The best feature value in feature-1 is " + str(bestfeature_1value))
print("The best feature value in feature-2 is " + str(bestfeature_2value))

The best feature value in feature-1 is 0.5588739367000624
The best feature value in feature-2 is 0.41734727680823214


In [19]:
comparebestfeature(bestfeature_1value,bestfeature_2value)

feature-1 gives best split with feature value 0.5588739367000624  and with best information gain of  0.6854247838416642


In [21]:
dfupdate=df_train.sample(10)
dfupdate.y=1
df_train.update(dfupdate)
update_list = dfupdate.index.tolist() # from  cᴏʟᴅsᴘᴇᴇᴅ  :)
df_train

,x1,x2,y
0,-0.219607,2.041501,0.0
1,-0.456096,2.196494,0.0
2,-0.130347,2.312636,0.0
3,-0.699596,1.778726,0.0
4,-1.180585,1.846638,0.0
...,...,...,...
131,1.796223,0.246648,1.0
132,2.192686,-0.349386,1.0
133,1.279821,-0.734666,1.0
134,2.140999,-0.148016,1.0


In [22]:
actualentropy=totentropy(df_train)

In [39]:
#actualentropy

0.6931471805599453

In [23]:
actualentropy

0.6904414521588527

In [24]:
dft1=df_train

In [25]:
feature_1=pd.DataFrame(dft1,columns=['x1','y']).sort_values(by = 'x1')
feature_1_df=pd.DataFrame(dft1,columns=['x1','y']).sort_values(by = 'x1').iloc[::7]
feature_1_infogain=informationgain(feature_1,feature_1_df)

In [26]:
feature_2=pd.DataFrame(dft1,columns=['x2','y']).sort_values(by = 'x2')
feature_2_df=pd.DataFrame(dft1,columns=['x2','y']).sort_values(by = 'x2').iloc[::7]
feature_2=feature_2.rename(columns={'x2': 'x1'})  
feature_2_df=feature_2_df.rename(columns={'x2': 'x1'})  
feature_2_infogain=informationgain(feature_2,feature_2_df)


In [27]:
bestfeature_1value=findbestfeaturevalue(feature_1_infogain)

In [28]:
bestfeature_2value=findbestfeaturevalue(feature_2_infogain)

In [40]:
print("The best feature value in feature-1 is " + str(bestfeature_1value))
print("The best feature value in feature-2 is " + str(bestfeature_2value))

The best feature value in feature-1 is 0.5588739367000624
The best feature value in feature-2 is 0.41734727680823214


In [41]:

comparebestfeature(bestfeature_1value,bestfeature_2value)


feature-1 gives best split with feature value 0.5588739367000624  and with best information gain of  0.651858004466276
